### Imports & Model Definition

In [1]:
import gurobipy as gp
from gurobipy import GRB

opt_mod = gp.Model(name="linear program")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-28


#### Data

In [2]:
ncf = [150, 100, -200, 200, -50, -300] # net cash_flow

n = 5 # xj : amount drawn from the line of credit in month j, for j = 1, . . . , 5
m = 3 # yj : amount of commercial paper issued in month j, for j = 1, . . . , 3
p = 6 # zj : excess funds in month j, for j = 1, . . . , 6.

# A = [[10, 10, 10, 10, 10, 110],
#        [7, 7, 7, 7, 7, 107],
#        [8, 8, 8, 8, 8, 108],
#        [6, 6, 6, 6, 106, 0],
#        [7, 7, 7, 7, 107, 0],
#        [5, 5, 5, 105, 0, 0],
#        [10, 10, 110, 0, 0, 0],
#        [8, 8, 108, 0, 0, 0],
#        [7, 107, 0, 0, 0, 0],
#        [100, 0, 0, 0, 0, 0]
# ]

#### Define decision variables

In [3]:
x = opt_mod.addVars(n,
                    name="x",
                    vtype=GRB.CONTINUOUS,
                    lb = 0, 
                    ub = 100,
)
x

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

In [4]:
y = opt_mod.addVars(m,
                    name="y",
                    vtype=GRB.CONTINUOUS,
                    lb = 0
)
y

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>}

In [5]:
z = opt_mod.addVars(p,
                    name="z",
                    vtype=GRB.CONTINUOUS,
                    lb = 0
)
z

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>}

#### Add Constraints

In [6]:
con1 = opt_mod.addConstr(x[0]       + y[0]                                - z[0] == ncf[0])
con2 = opt_mod.addConstr(x[1]       + y[1]      - 1.01*x[0] + 1.003*z[0]  - z[1] == ncf[1])
con3 = opt_mod.addConstr(x[2]       + y[2]      - 1.01*x[1] + 1.003*z[1]  - z[2] == ncf[2])
con4 = opt_mod.addConstr(x[3]       -1.02*y[0]  - 1.01*x[2] + 1.003*z[2]  - z[3] == ncf[3])
con5 = opt_mod.addConstr(x[4]       -1.02*y[1]  - 1.01*x[3] + 1.003*z[3]  - z[4] == ncf[4])
con6 = opt_mod.addConstr(           -1.02*y[2]  - 1.01*x[4] + 1.003*z[4]  - z[5] == ncf[5])




In [7]:
# for j in range(0, p):
#     if j == 0:
#         opt_mod.addConstr(x[j] + y[j] - z[j] == ncf[j])
#     elif j == 5:
#         opt_mod.addConstr(-1.02*y[2] - 1.01*x[4] + 1.003*z[4] - z[5] == ncf[j])
#     else:
#         if j == 3 or j == 4:
#             opt_mod.addConstr(x[j] - 1.02*y[j-3] - 1.01*x[j-1] + 1.003*z[j-1] - z[j] == ncf[j])
#         elif j == 1 or j == 2:
#             opt_mod.addConstr(x[j] + y[j] - 1.01*x[j-1] + 1.003*z[j-1] - z[j] == ncf[j])


### Set objective Function

In [8]:
opt_mod.setObjective(z[5], GRB.MAXIMIZE)

### Solve the model and output the solution

In [9]:
# opt_mod.setParam("OutputFlag", False)
opt_mod.optimize()
print(f"Optimization is done. Objective Function Value: {opt_mod.ObjVal}")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 14 columns and 27 nonzeros
Model fingerprint: 0x9745b9e3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+02, 1e+02]
  RHS range        [5e+01, 3e+02]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 12 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   2.944118e+30   8.000000e+00      0s
       6    9.2496949e+01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.03 seconds (0.00 work units)
Optimal objective  9.249694915e+01
Optimization is done. Objective Function Value: 92.49694915254233


In [10]:
for v in opt_mod.getVars():
    print(f"{v.varName}:{v.x}")

x[0]:0.0
x[1]:50.98039215686275
x[2]:0.0
x[3]:0.0
x[4]:0.0
y[0]:150.0
y[1]:49.01960784313725
y[2]:203.4343635759389
z[0]:0.0
z[1]:0.0
z[2]:351.9441674975075
z[3]:0.0
z[4]:0.0
z[5]:92.49694915254233


In [11]:
opt_mod.write("imunization.sol")

In [12]:
opt_mod.write("imunization.lp")
